In [20]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime

In [10]:
import spacy
nlp = spacy.load('en') # this should take some time like 10s to load

In [12]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [3]:
test_loc = 'test_dump.bin'
train_loc = 'train_dump.bin'

In [8]:
def clean(doc):
    b = [x for x in doc if not x.is_punct]
    a = [x.lemma_ if not x.lemma_ == '-PRON-' else x.text for x in b ]
    return nlp(u''.join([x + ' ' for x in a if x not in stop]))

In [13]:
from spacy.tokens.doc import Doc
test_docs = []
train_docs = []
i = 0
#print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open(train_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        if i%100000 == 0: print i, datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        train_docs.append(clean(Doc(nlp.vocab).from_bytes(byte_string)))
        i += 1

0 2017-05-14 22:33:39
100000 2017-05-14 22:34:45
200000 2017-05-14 22:36:09
300000 2017-05-14 22:37:17
400000 2017-05-14 22:38:21
500000 2017-05-14 22:39:30
600000 2017-05-14 22:40:40
700000 2017-05-14 22:41:57
800000 2017-05-14 22:43:02


# Load model for WMD

In [15]:
from gensim.models import word2vec

model = word2vec.Word2Vec.load("saved_vec_mod.kf")

In [18]:
def vec_diff(doc1,doc2):
    a = doc1.vector
    b = doc2.vector
    return a - b

def wmdist(sent1,sent2):
    """This requires that the parameters @sent1 @sent2 are lists of strings ideally
    lemmatized and cleaned sentences from above"""
    return model.wmdistance(sent1, sent2)

In [19]:
d = []
ct = 0
for q1,q2 in [(train_docs[2*i],train_docs[2*i+1]) for i in range(len(train_docs)/2)]:
    d.append({
              'sysim': q1.similarity(q2), # spacy sim
#              'netsim':  wn_sim(q1c,q2c), #float sim by wordnet
              'wmd': wmdist([x.text for x in q1], [x.text for x in q2]), # returns float dist
              'vectdiff': vec_diff(q1,q2) # 300 dim vect
    })
    if ct % 10000 == 0: print ct, datetime.now().strftime('%H:%M:%S')
    ct = ct+1    
features = pd.DataFrame(d)

0 23:06:52
10000 23:07:33
20000 23:07:58
30000 23:08:23
40000 23:08:39
50000 23:08:57
60000 23:09:13
70000 23:09:29
80000 23:09:44
90000 23:10:35
100000 23:10:55
110000 23:11:18
120000 23:11:35
130000 23:12:16
140000 23:12:34
150000 23:12:52
160000 23:13:15
170000 23:13:37
180000 23:13:59
190000 23:14:19
200000 23:15:36
210000 23:16:00
220000 23:16:19
230000 23:16:36
240000 23:16:52
250000 23:17:09
260000 23:17:27
270000 23:17:43
280000 23:17:58
290000 23:18:16
300000 23:18:36
310000 23:18:54
320000 23:19:12
330000 23:20:22
340000 23:20:39
350000 23:20:59
360000 23:21:18
370000 23:21:36
380000 23:21:54
390000 23:22:14
400000 23:22:29


NameError: name 'pd' is not defined

In [21]:
features = pd.DataFrame(d)

In [25]:
pos = []
neg = []
for idx, row in features.iterrows():
#     if idx == 10: break
    if row[''] row['wmd']

2.58744273093
10.5397637518
9.05921030816
19.3803418145
11.7262981678
7.01693150069
18.2013025203
4.1587520855
0.0
7.4102006765


# Joke Submission

In [38]:
with open('../data/charliesub.csv', 'w') as f:
    f.write("test_id,is_duplicate\n")
    for i in range(2345796):
        f.write(str(i)+",.17426\n")
f.close()

# Understanding log loss

In [35]:
def logloss(py,px):
    return -py*np.log(px) - (1-py)*np.log(1-px)

py = .5
px = .5
print logloss(px,py)

0.69314718056


In [36]:
np.log(.75)

-0.2876820724517809